# <div align="center">Fast R-CNN</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

R-CNN is slow in training & inference. We have 2,000 proposals which each of them needed to be processed by a CNN to extract features, and therefore, R-CNN will repeat the ConvNet 2,000 times to extract features.

<img src='asset/7_3/1.png'>

Hence, instead of converting 2,000 regions into the corresponding features maps, we convert the whole image once.

<img src='asset/7_3/2.png'>

Fast R-CNN is proposed as a single model instead of a pipeline to learn and output regions and classifications directly.

It’s primary computation steps are described below:

* Compared to an R-CNN model, a Fast R-CNN model uses the entire image as the CNN input for feature extraction, rather than each proposed region. Moreover, this network is generally trained to update the model parameters. As the input is an entire image, the CNN output shape is  ***1×c×h1×w1*** .


* Assuming selective search generates  ***n***  proposed regions, their different shapes indicate regions of interests (RoIs) of different shapes on the CNN output. Features of the same shapes must be extracted from these RoIs (here we assume that the height is  h2  and the width is  w2 ). Fast R-CNN introduces RoI pooling, which uses the CNN output and RoIs as input to output a concatenation of the features extracted from each proposed region with the shape  ***n×c×h2×w2*** .


* A fully connected layer is used to transform the output shape to  n×d , where  d  is determined by the model design.


* During category prediction, the shape of the fully connected layer output is again transformed to  ***n×q***  and we use softmax regression ( q  is the number of categories). During bounding box prediction, the shape of the fully connected layer output is again transformed to  ***n×4*** . This means that we predict the category and bounding box for each proposed region.

<img src='asset/7_3/3.png'>

The RoI pooling layer in Fast R-CNN is somewhat different from the pooling layers we have discussed before. In a normal pooling layer, we set the pooling window, padding, and stride to control the output shape. In an RoI pooling layer, we can directly specify the output shape of each region, such as specifying the height and width of each region as  h2,w2 . Assuming that the height and width of the RoI window are  h  and  w , this window is divided into a grid of sub-windows with the shape  h2×w2 . The size of each sub-window is about  (h/h2)×(w/w2) . The sub-window height and width must always be integers and the largest element is used as the output for a given sub-window. This allows the RoI pooling layer to extract features of the same shape from RoIs of different shapes.

Here we select an  3×3  region as an RoI of the  4×4  input. For this RoI, we use a  2×2  RoI pooling layer to obtain a single  2×2  output. When we divide the region into four sub-windows, they respectively contain the elements 0, 1, 4, and 5 (5 is the largest); 2 and 6 (6 is the largest); 8 and 9 (9 is the largest); and 10

<img src='asset/7_3/4.png'>

So Instead of extracting features for each image patch from scratch, we use a feature extractor (a CNN) to extract features for the whole image first. We also use an external region proposal method, like the selective search, to create ROIs which later combine with the corresponding feature maps to form patches for object detection. We warp the patches to a fixed size using ROI pooling and feed them to fully connected layers for classification and localization (detecting the location of the object). By not repeating the feature extractions, Fast R-CNN cuts down the process time significantly.

<img src='asset/7_3/5.jpg'>

Here is the network flow:

<img src='asset/7_3/6.jpg'>

In the pseudo-code below, the expensive feature extraction is moving out of the for-loop, a significant speed improvement since it was executed for all 2000 ROIs. Fast R-CNN is 10x faster than R-CNN in training and 150x faster in inferencing.

In [1]:
feature_maps = process(image)
ROIs = region_proposal(image)
for ROI in ROIs
    patch = roi_pooling(feature_maps, ROI)
    results = detector2(patch)

One major takeaway for Fast R-CNN is that the whole network (the feature extractor, the classifier, and the boundary box regressor) are trained end-to-end with multi-task losses (classification loss and localization loss). This improves accuracy.

### A little bit more information about ROI Pooling

Because Fast R-CNN uses fully connected layers, we apply ROI pooling to warp the variable size ROIs into in a predefined size shape.

Let’s simplify the discussion by transforming 8 × 8 feature maps into a predefined 2 × 2 shape.

* Top left below: our feature maps.


* Top right: we overlap the ROI (blue) with the feature maps.


* Bottom left: we split ROIs into the target dimension. For example, with our 2×2 target, we split the ROIs into 4 sections with similar or equal sizes.


* Bottom right: find the maximum for each section and the result is our warped feature maps.

<img src='asset/7_3/7.png'>

So we get a 2 × 2 feature patch that we can feed into the classifier and box regressor.

# <div align="center">Loss Function</div>
---------------------------------------------------------------------

As we said Fast R-CNN improves the training procedure by unifying three independent models into one jointly trained framework and increasing shared computation results. Instead of extracting CNN feature vectors independently for each region proposal, this model aggregates them into one CNN forward pass over the entire image and the region proposals share this feature matrix. Then the same feature matrix is branched out to be used for learning the object classifier and the bounding-box regressor. In conclusion, computation sharing speeds up R-CNN.

The model is optimized for a loss combining two tasks (classification + localization):

<img src='asset/7_3/10.png'>

<img src='asset/7_3/11.png'>